In [49]:
import os
import tarfile
import threading
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from azure.storage.blob import BlobServiceClient

# Setup Azure Blob Storage connection
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

class DownloadHandler(FileSystemEventHandler):
    def on_moved(self, event):
        print(event.src_path, "moved to", event.dest_path)
        if event.dest_path.endswith('.tar'):
            threading.Thread(target=self.unpack_and_upload, args=(event.dest_path,)).start()

    def unpack_and_upload(self, tar_path):
        extract_path = os.path.splitext(tar_path)[0]
        os.makedirs(extract_path, exist_ok=True)
        try:
            with tarfile.open(tar_path, 'r') as tar:
                tar.extractall(path=extract_path)
                print(f"Extracted {tar_path} to {extract_path}")
                self.upload_directory_to_blob(extract_path, os.path.basename(extract_path))
        except tarfile.TarError as e:
            print(f"Failed to extract {tar_path}: {e}")

    def upload_directory_to_blob(self, directory, folder_name):
        uploaded_count = 0
        total_files = 0
        for root, dirs, files in os.walk(directory):
            file_name = os.path.basename(root)
            for file in files:
                if file.endswith('.png'):
                    total_files += 1
                    file_path = os.path.join(root, file)
                    upload_path = f"samples/{folder_name}/{file_name}.png"
                    threading.Thread(target=self.upload_png_to_blob, args=(upload_path, file_path, file)).start()
                    uploaded_count += 1
        print(f"Uploaded {uploaded_count} out of {total_files} PNG files from {folder_name}")

    def upload_png_to_blob(self, upload_path, file_path, file_name):
        blob_name = upload_path
        try:
            blob_client = container_client.get_blob_client(blob_name)
            with open(file_path, 'rb') as file_data:
                blob_client.upload_blob(file_data, overwrite=True)
                print(f"Uploaded: {file_name} to {blob_name}")
        except Exception as e:
            print(f"Failed to upload {file_name} to {blob_name}: {e}")

def start_monitoring(path):
    event_handler = DownloadHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()
    print(f"Monitoring {path} for changes...")
    try:
        while True:
            threading.Event().wait(1)  # Sleeps to reduce CPU usage
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

if __name__ == '__main__':
    download_dir
    # Path to the directory to monitor
    download_dir = '/Volumes/SSD'
    start_monitoring(download_dir)

Monitoring /Volumes/SSD for changes...
/Volumes/SSD/val_images.tar.download/val_images.tar moved to /Volumes/SSD/val_images.tar
Extracted /Volumes/SSD/val_images.tar to /Volumes/SSD/val_images
Uploaded: naip.png to samples/val_images/26.649291999602085_-98.4511290001909.png
Uploaded: naip.png to samples/val_images/26.537475767253923_-81.79887711677804.png
Uploaded: naip.png to samples/val_images/30.1424_-85.6205.png
Uploaded: naip.png to samples/val_images/26.09679_-80.126222.png
Uploaded: naip.png to samples/val_images/32.009435000334264_-93.66506700008868.png
Uploaded: naip.png to samples/val_images/27.257436_-98.936989.png
Uploaded: naip.png to samples/val_images/25.5454_-80.3458.pngUploaded: naip.png to samples/val_images/33.68573761831119_-117.77852897416452.png

Uploaded: naip.png to samples/val_images/33.798068_-118.238409.png
Uploaded: naip.png to samples/val_images/33.370790596951274_-87.01907892711797.pngUploaded: naip.png to samples/val_images/33.81716457924358_-118.22643481

Exception in thread Thread-1119 (unpack_and_upload):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/cz/v3jc1bk16yj2kjzwjtw6mhmm0000gn/T/ipykernel_89632/3818933370.py", line 27, in unpack_and_upload
  File "/var/folders/cz/v3jc1bk16yj2kjzwjtw6mhmm0000gn/T/ipykernel_89632/3818933370.py", line 41, in upload_directory_to_blob
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 928, in start
    _start_new_thread(self._bootstrap, ())
RuntimeError: can't start new thread


Uploaded: naip.png to samples/train_images_2/27.18578648683509_-82.50182305875222.png
Uploaded: naip.png to samples/train_images_2/31.622956221790805_-93.78514422594148.png
Uploaded: naip.png to samples/train_images_2/30.167901000010204_-93.62170099981932.png
Uploaded: naip.png to samples/train_images_2/30.921719518245848_-95.82655157182006.png
Uploaded: naip.png to samples/train_images_2/26.417440688132_-82.04383391409633.png
Uploaded: naip.png to samples/train_images_2/30.22349119182485_-87.82766213738257.png
Uploaded: naip.png to samples/train_images_2/30.79145264133774_-102.7081565786803.png
Uploaded: naip.png to samples/train_images_2/30.916924337005646_-98.6486828950664.png
Uploaded: naip.png to samples/train_images_2/30.595222738352565_-83.18844235367602.png
Uploaded: naip.png to samples/train_images_2/30.957543696348367_-91.63473535543704.png
Uploaded: naip.png to samples/train_images_2/30.88443243109988_-96.43353897575544.png
Uploaded: naip.png to samples/train_images_2/27.146

Exception in thread Thread-3699 (unpack_and_upload):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/cz/v3jc1bk16yj2kjzwjtw6mhmm0000gn/T/ipykernel_89632/3818933370.py", line 27, in unpack_and_upload
  File "/var/folders/cz/v3jc1bk16yj2kjzwjtw6mhmm0000gn/T/ipykernel_89632/3818933370.py", line 41, in upload_directory_to_blob
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 928, in start
    _start_new_thread(self._bootstrap, ())
RuntimeError: can't start new thread


Uploaded: naip.png to samples/train_images_3/29.32444556611828_-90.3068886957912.png
Uploaded: naip.png to samples/train_images_3/28.76165_-81.738.png
Uploaded: naip.png to samples/train_images_3/30.214192576809204_-89.0815701651292.pngUploaded: naip.png to samples/train_images_3/28.743230035334108_-80.81679593537802.png

Uploaded: naip.png to samples/train_images_3/28.750614276024336_-95.60355795187948.png
Uploaded: naip.png to samples/train_images_3/26.45627094027608_-81.94404044740583.png
Uploaded: naip.png to samples/train_images_3/30.40670405150829_-85.12551273996945.png
Uploaded: naip.png to samples/train_images_3/30.278000255452085_-92.58377239724908.png
Uploaded: naip.png to samples/train_images_3/30.34628262741022_-88.39664903901229.png
Uploaded: naip.png to samples/train_images_3/30.381_-86.879.png
Uploaded: naip.png to samples/train_images_3/30.27428436377995_-89.35156332989109.png
Uploaded: naip.png to samples/train_images_3/30.348645617913448_-86.23987222773046.png
Uploade

Exception in thread Thread-5467 (unpack_and_upload):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/cz/v3jc1bk16yj2kjzwjtw6mhmm0000gn/T/ipykernel_89632/3818933370.py", line 27, in unpack_and_upload
  File "/var/folders/cz/v3jc1bk16yj2kjzwjtw6mhmm0000gn/T/ipykernel_89632/3818933370.py", line 41, in upload_directory_to_blob
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 928, in start
    _start_new_thread(self._bootstrap, ())
RuntimeError: can't start new thread


Uploaded: naip.png to samples/train_images_1/29.932_-95.38.pngUploaded: naip.png to samples/train_images_1/30.010358_-91.106768.png

Uploaded: naip.png to samples/train_images_1/30.26719514039885_-85.6610197170504.png
Uploaded: naip.png to samples/train_images_1/29.95430346298288_-90.36629763523882.png
Uploaded: naip.png to samples/train_images_1/31.274046044687488_-93.59802865480655.png
Uploaded: naip.png to samples/train_images_1/29.875_-95.394.png
Uploaded: naip.png to samples/train_images_1/30.03970477514829_-82.73148528692883.png
Uploaded: naip.png to samples/train_images_1/29.9958_-93.3419.png
Uploaded: naip.png to samples/train_images_1/29.816958093679723_-88.88920755389505.png
Uploaded: naip.png to samples/train_images_1/30.218546838527825_-81.87749555806488.png
Uploaded: naip.png to samples/train_images_1/29.89987570352061_-98.23601031756208.png
Uploaded: naip.png to samples/train_images_1/29.866798404439606_-89.67908705971621.png
Uploaded: naip.png to samples/train_images_1/2

In [4]:
import os
import tarfile
import threading
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from azure.storage.blob import BlobServiceClient

#check number ob blobs in folder
# Setup Azure Blob Storage connection
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

folders=['test_images','val_images','train_images_1','train_images_2', 'train_images_3']
# Specify the folder prefix (e.g., "samples/")
prefix = "samples/"
for folder in folders: 
    # List all blobs in the container
    folder_prefix=f"{prefix}{folder}/"
    blobs = container_client.list_blobs(name_starts_with=folder_prefix)

    # Count the blobs that start with the specified folder prefix
    blob_count = sum(1 for _ in blobs)  # Generator comprehension to count blobs

    print(f"Number of blobs in '{folder_prefix}': {blob_count}")


Number of blobs in 'samples/test_images/': 1018
Number of blobs in 'samples/val_images/': 515
Number of blobs in 'samples/train_images_1/': 2716
Number of blobs in 'samples/train_images_2/': 28415
Number of blobs in 'samples/train_images_3/': 2115
